# Import librerie e definizione funzioni d'utilità

In [ ]:
import functions as fn

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import geopandas as gpd

In [ ]:
dataset_name = "govuk"
if not os.path.exists("..\\Plot"):
    os.mkdir("..\\Plot")
if not os.path.exists("..\\Plot\\" + dataset_name):
    os.mkdir("..\\Plot\\" + dataset_name)
if not os.path.exists("..\\Plot\\" + dataset_name + "\\tempi"):
    os.mkdir("..\\Plot\\" + dataset_name + "\\tempi")
if not os.path.exists("..\\DatasetParsed"):
    os.mkdir("..\\DatasetParsed")
if not os.path.exists("..\\DatasetParsed\\" + dataset_name):
    os.mkdir("..\\DatasetParsed\\" + dataset_name)

In [ ]:
counter = 0
def make_path(dataset, file_name, other = ""):
    global counter
    counter += 1
    return "..\\Plot\\" + dataset + "\\" + other + str(counter) + "-" + file_name

# Estrazione dati tabella aziende

In [ ]:
dataset_govuk = "..\\Datasets\\" + dataset_name + "\\govuk-aziende-dataset.csv"
df_govuk = pd.read_csv(dataset_govuk, encoding='cp1252')
df_time = df_govuk[["URL", "Tempo richiesta", "Tempo estrazione Name", "Tempo estrazione Company ID", "Tempo estrazione Company Status", "Tempo estrazione Company Type", "Tempo estrazione Reg/Inc Date", "Tempo estrazione Diss Date", "Tempo estrazione Address"]]
df_govuk = df_govuk[["URL", "ID", "Name", "Company ID", "Company Status", "Company Type", "Registration Date", "Incorporation Date", "Dissolution Date", "Office Address"]]
print("Numero entry tabella originale: ", len(df_govuk))

In [ ]:
df_govuk.to_csv("..\\DatasetParsed\\" + dataset_name + "\\" + dataset_name + "-aziende.csv", index=False)
df_govuk.head()

In [ ]:
info_dataset = fn.extract_data_from_df(df_govuk)
display(pd.DataFrame(info_dataset, index=["valore"]).T)

### Statistiche celle

In [ ]:
keys = ["celle totali", "celle nulle"]
values = [
    info_dataset["numero celle"], 
    info_dataset["numero celle nulle"]
]
title = "Confronto numero celle totali e vuote"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
keys = ["celle totali", "celle str", "celle float", "celle int"]
values = [
    info_dataset["numero celle"], 
    info_dataset["numero celle str"],
    info_dataset["numero celle float"],
    info_dataset["numero celle int"]
]
title = "Confronto tipo celle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

### Statistiche righe

In [ ]:
keys = ["righe totali", "righe con celle nulle"]
values = [
    info_dataset["numero righe"], 
    info_dataset["numero righe con celle nulle"]
]
title = "Confronto numero righe totali e nulle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

### Statistiche colonne

In [ ]:
keys = ["colonne totali", "colonne con celle nulle"]
values = [
    info_dataset["numero colonne"], 
    info_dataset["numero colonne con celle nulle"]
]
title = "Confronto numero colonne totali e nulle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
keys = ["colonne totali", "colonne str", "colonne float", "colonne int"]
values = [
    info_dataset["numero colonne"], 
    info_dataset["numero colonne str"],
    info_dataset["numero colonne float"],
    info_dataset["numero colonne int"]
]
title = "Confronto tipo colonne"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
different_value_columns = {}
for key in df_govuk:
    different_value_columns[key] = [len(pd.unique(df_govuk[key])), df_govuk[key].isnull().sum()]
different_value_columns = dict(sorted(different_value_columns.items(), key=lambda item: item[1], reverse=True))

In [ ]:
keys = different_value_columns.keys()
values = [item[0] for item in different_value_columns.values()]
title = "Numero di valori distinti per colonna"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (8,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim, True, info_dataset["numero righe"])

In [ ]:
keys = different_value_columns.keys()
values = [item[1] for item in different_value_columns.values()]
title = "Numero di valori nulli per colonna"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (8,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim, True, info_dataset["numero righe"])

### Plot su mappe geografiche

In [ ]:
df_govuk = df_govuk[df_govuk['Office Address'].isnull() == False]
print("Filtro Office Address null: ", len(df_govuk))
caps = []
pattern = re.compile("^[A-Z]([A-Z]|[0-9]){4,6}")
for address in df_govuk["Office Address"]:
    token = address.split(" ")
    if pattern.match(token[-1]):
        caps.append(token[-1])
print("Filtro Office Address senza CAP: ", len(caps))

In [ ]:
file_postcodes = "..\\CAP\\" + dataset_name + "\\postcodes.csv"
postcodes = pd.read_csv(file_postcodes)
postcodes = postcodes[['Postcode', 'UK region']]
postcodes = postcodes.rename(columns={"Postcode": "postcode", "UK region": "region"})

##### Nazioni UK

In [ ]:
nation = []
for cap in caps:
    entry = postcodes.loc[postcodes.postcode.apply(lambda x: cap.startswith(x))]
    if len(entry.region.values) > 0:
        s = entry.region.values[0]
        if s not in ["Scotland", "Northern Ireland", "Wales"] and s != None:
            s = "England"
        nation.append(s)
print("Filtro CAP non presenti in csv (potrebbero essere NaN, per questo non sono presenti): ", len(nation))

In [ ]:
n_companies_by_nation = {}
res = []
[res.append(x) for x in nation if x not in res]
for c in res:
    n_companies_by_nation[c] = nation.count(c)

In [ ]:
fp = "..\\Mappe\\" + dataset_name + "\\UK\\uk.shp"
map_df = gpd.read_file(fp)
map_df["n_companies"] = np.nan
for i, row in map_df.iterrows():
    if row[2] in n_companies_by_nation:
        map_df.at[i, 'n_companies'] = n_companies_by_nation[row[2]]
map_df

In [ ]:
fn.map_plot(map_df, "n_companies", "ctry18nm", "geometry", 10, "Numero di aziende per nazione", make_path(dataset_name, "uk"))

##### Regioni inghilterra

In [ ]:
region = []
for cap in caps:
    entry = postcodes.loc[postcodes.postcode.apply(lambda x: cap.startswith(x))]
    if len(entry.region.values) > 0:
        region.append(entry.region.values[0])
print("Filtro CAP non presenti in csv: ", len(region))

In [ ]:
n_companies_by_region = {}
res = []
[res.append(x) for x in region if x not in res]
for c in res:
    n_companies_by_region[c] = region.count(c)

In [ ]:
fp = "..\\Mappe\\" + dataset_name + "\\Inghilterra\\inghilterra.shp"
map_df = gpd.read_file(fp)
map_df["n_companies"] = np.nan
for i, row in map_df.iterrows():
    if row[2] in n_companies_by_region:
        map_df.at[i, 'n_companies'] = n_companies_by_region[row[2]]
map_df

In [ ]:
fn.map_plot(map_df, "n_companies", "rgn17nm", "geometry", 10,"Numero di aziende per regione", make_path(dataset_name, "inghilterra"))

# Estrazione prestazioni

In [ ]:
# Tempo totale di esecuzione 1: 231.700044631958 (secondi), 420 entry
# Tempo totale di esecuzione 2: 231.72814297676086 (secondi), 440 entry
# Tempo totale di esecuzione 4: 250.68748903274536 (secondi), 420 entry

In [ ]:
df_time.head()

In [ ]:
df_time.describe()

In [ ]:
new_df_time = df_time.describe()
del new_df_time["Tempo richiesta"]

In [ ]:
keys = ["Tempo richiesta", "Tempo estrazione dettagli"]
values = [df_time.describe()["Tempo richiesta"]["mean"], new_df_time.sum(axis="columns")["mean"]]
values = np.array(values) * 1000
title = "Confronto tempo medio (ms) request vs tempo medio (ms) estrazione dettagli"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
keys = list(new_df_time.loc["mean"].keys())
values = list(new_df_time.loc["mean"])
values = np.array(values) * 1000000
title = "Tempi medi (\u03BCs) di estrazione dei valori (solo xpath)"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
keys = list(new_df_time.loc["max"].keys())
values = list(new_df_time.loc["max"])
values = np.array(values) * 1000
title = "Tempo massimo (ms) di estrazione dei valori (solo xpath)"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)